In [1]:
import os
from dotenv import load_dotenv
import boto3
from pyspark.sql import SparkSession
import pandas as pd
from pathlib import Path
from io import BytesIO

In [2]:
# carregar variáveis do .env
load_dotenv()

True

In [3]:
try:
    # Inicialização do client da AWS
    s3 = boto3.client(
        's3',
        aws_access_key_id=os.getenv("AWS_ACCESS_KEY_ID"),
        aws_secret_access_key=os.getenv("AWS_SECRET_ACCESS_KEY"),
        aws_session_token=os.getenv("AWS_SESSION_TOKEN"),
        region_name=os.getenv("AWS_REGION")
    )

    print(f"✅ Conexão com AWS estabelecida!")

    # Busca dos arquivos do S3
    objetos = s3.list_objects_v2(
        Bucket=os.getenv("S3_BUCKET_NAME"),
        Prefix="raw/"
    )

    arquivos_bucket = []

    # separar arquivo de dicionario dos arquivos de dados
    if 'Contents' in objetos:
        for obj in objetos['Contents']:
            if obj['Size'] > 0:
                arquivos_bucket.append(obj['Key'])
    else:
        print("⚠️ Nenhum arquivo encontrado na pasta 'raw'")

    if len(arquivos_bucket) > 0:
        print(f"✅ Arquivos de CSVs encontrados! {arquivos_bucket}")

except Exception as e:
    print(f"❌ Erro de conexão: {e}")

✅ Conexão com AWS estabelecida!
✅ Arquivos de CSVs encontrados! ['raw/Dicionario_PNAD_COVID_112020_20220621.xls', 'raw/PNAD_COVID_092020.csv', 'raw/PNAD_COVID_102020.csv', 'raw/PNAD_COVID_112020.csv']


In [4]:
spark = (
    SparkSession.builder
      .appName("IBGE_Bronze")
      .config("spark.driver.memory", "4g")
      .config("spark.executor.memory", "4g")
      .config("spark.jars.packages", "org.postgresql:postgresql:42.7.3")
      .config("spark.hadoop.fs.s3a.aws.region", os.getenv('AWS_REGION'))
      .getOrCreate()
)

:: loading settings :: url = jar:file:/opt/homebrew/lib/python3.11/site-packages/pyspark/jars/ivy-2.5.3.jar!/org/apache/ivy/core/settings/ivysettings.xml
Ivy Default Cache set to: /Users/hgirardi/.ivy2.5.2/cache
The jars for the packages stored in: /Users/hgirardi/.ivy2.5.2/jars
org.postgresql#postgresql added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-5ccad24a-f864-4b65-916e-60971a08c972;1.0
	confs: [default]
	found org.postgresql#postgresql;42.7.3 in central
	found org.checkerframework#checker-qual;3.42.0 in central
:: resolution report :: resolve 57ms :: artifacts dl 2ms
	:: modules in use:
	org.checkerframework#checker-qual;3.42.0 from central in [default]
	org.postgresql#postgresql;42.7.3 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	------------------

----------------------------------------
Exception occurred during processing of request from ('127.0.0.1', 51694)
Traceback (most recent call last):
  File "/opt/homebrew/Cellar/python@3.11/3.11.13/Frameworks/Python.framework/Versions/3.11/lib/python3.11/socketserver.py", line 317, in _handle_request_noblock
    self.process_request(request, client_address)
  File "/opt/homebrew/Cellar/python@3.11/3.11.13/Frameworks/Python.framework/Versions/3.11/lib/python3.11/socketserver.py", line 348, in process_request
    self.finish_request(request, client_address)
  File "/opt/homebrew/Cellar/python@3.11/3.11.13/Frameworks/Python.framework/Versions/3.11/lib/python3.11/socketserver.py", line 361, in finish_request
    self.RequestHandlerClass(request, client_address, self)
  File "/opt/homebrew/Cellar/python@3.11/3.11.13/Frameworks/Python.framework/Versions/3.11/lib/python3.11/socketserver.py", line 755, in __init__
    self.handle()
  File "/opt/homebrew/lib/python3.11/site-packages/pyspark/ac

In [5]:
print(spark.version)
print(f"Versão Scala: {spark.sparkContext.version}")
print(f"Detalhes: {spark.sparkContext.applicationId}")

4.0.1
Versão Scala: 4.0.1
Detalhes: local-1758842756145


In [6]:
import psycopg2

# Dados de conexão com o Postgres
jdbc_url = f"jdbc:postgresql://{os.getenv('POSTGRES_HOST')}:{os.getenv('POSTGRES_PORT')}/{os.getenv('POSTGRES_DATABASE')}"
connection_properties = {
    "user": os.getenv('POSTGRES_USER'),
    "password": os.getenv('POSTGRES_PASSWORD'),
    "driver": "org.postgresql.Driver"
}

""" Criação do schema Bronze no banco de dados """

print(f"== Iniciando processo de criação do schema ==")
create_schema_sql = "CREATE SCHEMA IF NOT EXISTS bronze;"

try:

    conn = psycopg2.connect(
        host=os.getenv('POSTGRES_HOST'),
        port=os.getenv('POSTGRES_PORT'),
        database=os.getenv('POSTGRES_DATABASE'),
        user=connection_properties['user'],
        password=connection_properties['password']
    )
    print(f"✅ Connexão estabelecida!")

    with conn.cursor() as cursor:
        cursor.execute(create_schema_sql)
        conn.commit()
        print("✅ SQL executado!")
    

except psycopg2.Error as e:
    print(f"❌ Erro ao executar SQL: {e}")
    raise
except Exception as e:
    print(f"❌ Erro genérico: {e}")
    raise
finally:
    if 'conn' in locals() and conn:
        conn.close()
        print(f"✅ Connexão fechada!")

print(f"== Processo finalizado ==")


== Iniciando processo de criação do schema ==
✅ Connexão estabelecida!
✅ SQL executado!
✅ Connexão fechada!
== Processo finalizado ==


In [9]:
def read_file(arquivo):
    """ 
    Leitura do arquivo no Bucket para retornar o DF
    Criação do DF será feita de acordo com a extensão do arquivo 
    Extensões suportadas: .csv e .xls/.xlsx
    """
    extensao = Path(arquivo).suffix.lower()

    if extensao == '.csv':
        return spark.read.option("header", "true").option("inferSchema", "false").csv(f"s3a://{os.getenv('S3_BUCKET_NAME')}/{arquivo}")
    
    elif extensao in ['.xls', '.xlsx']:
        obj = s3.get_object(Bucket=os.getenv('S3_BUCKET_NAME'), Key=arquivo)
        
        df = pd.read_excel(BytesIO(obj['Body'].read()))
        df = df.reset_index() # Adiciona coluna 'index' com número sequencial
        df = df.astype(str)
              
        return spark.createDataFrame(df)

    else:
        raise ValueError(f"❌ Extensão não suportada: {extensao}")
    

In [10]:
from pyspark.sql.utils import AnalysisException
from py4j.protocol import Py4JJavaError

print("== Percorrendo os arquivos do Bucket para criação das tabelas no banco de dados ==")
print("")
for arquivo in arquivos_bucket:
    """Criação dinâmica das tabelas no Postgres"""
    
    nome_tabela = f"bronze.raw_{Path(arquivo).stem.lower()}"     
        
    print(f"  Arquivo: {arquivo}")
    print(f"  JDBC_URL: {jdbc_url}")
    print(f"  Nome da tabela: {nome_tabela}")
    print(f"  Postgres Propriedades: {connection_properties}")

    try:
        df = read_file(arquivo)
        print("")
        print(f"  Dados carregados no DF: {df.count()} linhas | {df.columns} colunas")
        df.show(10)
        print("")

        df.write.mode("overwrite").jdbc(jdbc_url, nome_tabela, properties=connection_properties)
        
        print(f"  ✅ Tabela {nome_tabela} criada com sucesso!")
        print("")
        print("  " + "-"*50)
        print("")

    except AnalysisException as e:
        print(f"  ❌ Erro de análise Spark (arquivo/schema): {e}")
        break

    except psycopg2.Error as e:
        print(f"  ❌ Erro psycopg2: {e}")
        break
    
    except Py4JJavaError as e:
        print(f"  ❌ Erro Py4JJavaError: {e}")
        break

    except Exception as e:
        print(f"  ⚠️ Erro genérico: {e}")

    finally:
        print("")
        print("== Criação das tabelas concluída == ")



== Percorrendo os arquivos do Bucket para criação das tabelas no banco de dados ==

  Arquivo: raw/Dicionario_PNAD_COVID_112020_20220621.xls
  JDBC_URL: jdbc:postgresql://tech-challenge-3-ibge.c18mok42i31i.us-east-1.rds.amazonaws.com:5432/tech-challenge-3-ibge
  Nome da tabela: bronze.raw_dicionario_pnad_covid_112020_20220621
  Postgres Propriedades: {'user': 'postgres', 'password': 'postgres', 'driver': 'org.postgresql.Driver'}

  Dados carregados no DF: 660 linhas | ['index', 'Dicionário das variáveis da PNAD COVID', 'Unnamed: 1', 'Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4', 'Unnamed: 5'] colunas
+-----+--------------------------------------+--------------------+----------+--------------------+----------+----------+
|index|Dicionário das variáveis da PNAD COVID|          Unnamed: 1|Unnamed: 2|          Unnamed: 3|Unnamed: 4|Unnamed: 5|
+-----+--------------------------------------+--------------------+----------+--------------------+----------+----------+
|    0|                        

  ✅ Tabela bronze.raw_dicionario_pnad_covid_112020_20220621 criada com sucesso!

  --------------------------------------------------


== Criação das tabelas concluída == 
  Arquivo: raw/PNAD_COVID_092020.csv
  JDBC_URL: jdbc:postgresql://tech-challenge-3-ibge.c18mok42i31i.us-east-1.rds.amazonaws.com:5432/tech-challenge-3-ibge
  Nome da tabela: bronze.raw_pnad_covid_092020
  Postgres Propriedades: {'user': 'postgres', 'password': 'postgres', 'driver': 'org.postgresql.Driver'}



  Dados carregados no DF: 387298 linhas | ['Ano', 'UF', 'CAPITAL', 'RM_RIDE', 'V1008', 'V1012', 'V1013', 'V1016', 'Estrato', 'UPA', 'V1022', 'V1023', 'V1030', 'V1031', 'V1032', 'posest', 'A001', 'A001A', 'A001B1', 'A001B2', 'A001B3', 'A002', 'A003', 'A004', 'A005', 'A006', 'A007', 'A008', 'A009', 'B0011', 'B0012', 'B0013', 'B0014', 'B0015', 'B0016', 'B0017', 'B0018', 'B0019', 'B00110', 'B00111', 'B00112', 'B00113', 'B002', 'B0031', 'B0032', 'B0033', 'B0034', 'B0035', 'B0036', 'B0037', 'B0041', 'B0042', 'B0043', 'B0044', 'B0045', 'B0046', 'B005', 'B006', 'B007', 'B008', 'B009A', 'B009B', 'B009C', 'B009D', 'B009E', 'B009F', 'B0101', 'B0102', 'B0103', 'B0104', 'B0105', 'B0106', 'B011', 'C001', 'C002', 'C003', 'C004', 'C005', 'C0051', 'C0052', 'C0053', 'C006', 'C007', 'C007A', 'C007B', 'C007C', 'C007D', 'C007E', 'C007E1', 'C007E2', 'C007F', 'C008', 'C009', 'C009A', 'C010', 'C0101', 'C01011', 'C01012', 'C0102', 'C01021', 'C01022', 'C0103', 'C0104', 'C011A', 'C011A1', 'C011A11', 'C011A12', '

+----+---+-------+-------+-----+-----+-----+-----+-------+---------+-----+-----+------+------------+------------+------+----+-----+------+------+------+----+----+----+----+----+----+----+----+-----+-----+-----+-----+-----+-----+-----+-----+-----+------+------+------+------+----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+----+----+----+----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+----+----+----+----+----+----+-----+-----+-----+----+----+-----+-----+-----+-----+-----+------+------+-----+----+----+-----+----+-----+------+--------+-----+------+------+-----+-----+-----+------+-------+--------+------+-------+-------+----+----+----+----+----+-----+-----+--------+-----+-----+-----+-----+-----+--------+-----+--------+-----+-----+-----+-----+----+-----+-----+-----+-----+----+-----+-----+------+------+------+------+------+-----+----+
| Ano| UF|CAPITAL|RM_RIDE|V1008|V1012|V1013|V1016|Estrato|      UPA|V1022|V1023| V1030|       V103

  ✅ Tabela bronze.raw_pnad_covid_092020 criada com sucesso!

  --------------------------------------------------


== Criação das tabelas concluída == 
  Arquivo: raw/PNAD_COVID_102020.csv
  JDBC_URL: jdbc:postgresql://tech-challenge-3-ibge.c18mok42i31i.us-east-1.rds.amazonaws.com:5432/tech-challenge-3-ibge
  Nome da tabela: bronze.raw_pnad_covid_102020
  Postgres Propriedades: {'user': 'postgres', 'password': 'postgres', 'driver': 'org.postgresql.Driver'}



  Dados carregados no DF: 380461 linhas | ['Ano', 'UF', 'CAPITAL', 'RM_RIDE', 'V1008', 'V1012', 'V1013', 'V1016', 'Estrato', 'UPA', 'V1022', 'V1023', 'V1030', 'V1031', 'V1032', 'posest', 'A001', 'A001A', 'A001B1', 'A001B2', 'A001B3', 'A002', 'A003', 'A004', 'A005', 'A006', 'A007', 'A008', 'A009', 'B0011', 'B0012', 'B0013', 'B0014', 'B0015', 'B0016', 'B0017', 'B0018', 'B0019', 'B00110', 'B00111', 'B00112', 'B00113', 'B002', 'B0031', 'B0032', 'B0033', 'B0034', 'B0035', 'B0036', 'B0037', 'B0041', 'B0042', 'B0043', 'B0044', 'B0045', 'B0046', 'B005', 'B006', 'B007', 'B008', 'B009A', 'B009B', 'B009C', 'B009D', 'B009E', 'B009F', 'B0101', 'B0102', 'B0103', 'B0104', 'B0105', 'B0106', 'B011', 'C001', 'C002', 'C003', 'C004', 'C005', 'C0051', 'C0052', 'C0053', 'C006', 'C007', 'C007A', 'C007B', 'C007C', 'C007D', 'C007E', 'C007E1', 'C007E2', 'C007F', 'C008', 'C009', 'C009A', 'C010', 'C0101', 'C01011', 'C01012', 'C0102', 'C01021', 'C01022', 'C0103', 'C0104', 'C011A', 'C011A1', 'C011A11', 'C011A12', '

  ✅ Tabela bronze.raw_pnad_covid_102020 criada com sucesso!

  --------------------------------------------------


== Criação das tabelas concluída == 
  Arquivo: raw/PNAD_COVID_112020.csv
  JDBC_URL: jdbc:postgresql://tech-challenge-3-ibge.c18mok42i31i.us-east-1.rds.amazonaws.com:5432/tech-challenge-3-ibge
  Nome da tabela: bronze.raw_pnad_covid_112020
  Postgres Propriedades: {'user': 'postgres', 'password': 'postgres', 'driver': 'org.postgresql.Driver'}



  Dados carregados no DF: 381438 linhas | ['Ano', 'UF', 'CAPITAL', 'RM_RIDE', 'V1008', 'V1012', 'V1013', 'V1016', 'Estrato', 'UPA', 'V1022', 'V1023', 'V1030', 'V1031', 'V1032', 'posest', 'A001', 'A001A', 'A001B1', 'A001B2', 'A001B3', 'A002', 'A003', 'A004', 'A005', 'A006', 'A006A', 'A006B', 'A007', 'A007A', 'A008', 'A009', 'B0011', 'B0012', 'B0013', 'B0014', 'B0015', 'B0016', 'B0017', 'B0018', 'B0019', 'B00110', 'B00111', 'B00112', 'B00113', 'B002', 'B0031', 'B0032', 'B0033', 'B0034', 'B0035', 'B0036', 'B0037', 'B0041', 'B0042', 'B0043', 'B0044', 'B0045', 'B0046', 'B005', 'B006', 'B007', 'B008', 'B009A', 'B009B', 'B009C', 'B009D', 'B009E', 'B009F', 'B0101', 'B0102', 'B0103', 'B0104', 'B0105', 'B0106', 'B011', 'C001', 'C002', 'C003', 'C004', 'C005', 'C0051', 'C0052', 'C0053', 'C006', 'C007', 'C007A', 'C007B', 'C007C', 'C007D', 'C007E', 'C007E1', 'C007E2', 'C007F', 'C008', 'C009', 'C009A', 'C010', 'C0101', 'C01011', 'C01012', 'C0102', 'C01021', 'C01022', 'C0103', 'C0104', 'C011A', 'C011A

  ✅ Tabela bronze.raw_pnad_covid_112020 criada com sucesso!

  --------------------------------------------------


== Criação das tabelas concluída == 


25/09/25 22:22:54 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 275219 ms exceeds timeout 120000 ms
25/09/25 22:22:54 WARN SparkContext: Killing executors is not supported by current scheduler.
25/09/25 22:22:59 ERROR Inbox: Ignoring error
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.SparkThreadUtils$.awaitResult(SparkThreadUtils.scala:53)
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:342)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRefByURI(RpcEnv.scala:102)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRef(RpcEnv.scala:110)
	at org.apache.spark.util.RpcUtils$.makeDriverRef(RpcUtils.scala:36)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.driverEndpoint$lzycompute(BlockManagerMasterEndpoint.scala:132)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.org$apache$spark$storage$BlockManagerMasterEndpoint$$